AGENT:DİĞER ARAÇLARA ETKİLEŞİME GEÇMEK İÇİN DİL MODELLERİNİ KULLANIRLAR.Yani agentlar alınacak aksiyonu kendileri seçerler.
Biz iki araç kullanıcaz; RETRİEVER VE SEARCH ENGİNE(Güncel bilgi gerektiren soruları cevaplandıracak)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

CREATİNG RETRİEVER CHAIN

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://blog.langchain.dev/langgraph/")
docs=loader.load()


In [ ]:
docs

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

Dil modelleri belirli sayıda token kabul ettiğinden dolayı metni dökümanlara ayırıp parçalamamamız
gerekicek.RecursiveCharacterTextSplitter sınıfını import edelim

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter()
documents=text_splitter.split_documents(docs)

In [ ]:
documents

Şimdi bu dökümanları vektör veri tabanına endeksleyelim

In [ ]:
from langchain_community.vectorstores import FAISS
vector=FAISS.from_documents(documents, embeddings)

Artık bu endekslenen verileri yakalayabiliriz

In [ ]:
retriever=vector.as_retriever()


In [ ]:
retriever.get_relevant_documents(
    "What is LangGraph?"
)[0]


İstediğimiz bilgileri yakalamak için şimdi ilk toolumuzu oluşturalım.CREAT RETRİEVER TOOL

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(
    retriever,
    "langhraph search",
    "Search for information about LangGraph.For any question\
        about LangGraph,you must use this tool!"
)

CREATİNG SEARCH TOOL : Search tool olarak tavily toolunu kullanacağız.Bu arama motoru özellikle yapay zeka agentları 
için geliştirilmiştir

In [ ]:
from langchain_community.tools.tavily_search import TavılySearchResults
search=TavılySearchResults()

In [ ]:
search.invoke("What is the weather in Istanul ?")

In [ ]:
tools=[retriever_tool,search]

In [ ]:
from langchain_openai import ChatOpenAI
model=ChatOpenAI(temperature=0)

In [ ]:
from langchain import hub
prompt=hub.pull("hwchase17/structured-chat-agent")

In [ ]:
prompt.messages

INITIALIZING THE AGENT

In [ ]:
from langchain.agents import create_openai_functions_agent
agent=create_openai_functions_agent(model,tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

EXECUTİNG THE AGENT

In [ ]:
agent_executor.invoke(
    {
        "input":"Hi!"
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input":"What is the weater in London!"
    }
)

In [ ]:
from langchain_core.messages import HumanMessage,AIMessage
chat_history=[
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes!"),
]

In [ ]:
agent_executor.invoke(
    {
        "chat_history":chat_history,
        "input":"Tell me how",
    }
    
)